In [2]:
import gradio as gr
import openai
import json
import os

In [7]:
%pip install pydrive

  Using cached PyDrive-1.3.1.tar.gz (987 kB)
  Preparing metadata (setup.py) ... done
  Using cached google_api_python_client-2.185.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached oauth2client-4.1.3-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached httplib2-0.31.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached google_auth-2.42.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_api_core-2.28.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached uritemplate-4.2.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached googleapis_common_protos-1.71.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
Using cached google_api_python_client-2.185.0-py3-non

In [8]:
import os
import mykeys


In [9]:
mykeys.setOsEnv()

아래 링크를 복사하여 웹 브라우저에 붙여넣으세요.
https://accounts.google.com/o/oauth2/auth?client_id=35726703810-4v13dfqmilhgv6shlc3cv9i3ktuh73j1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code
Authentication successful.
mykey => set HUGGINGFACEHUB_API_TOKEN is done
mykey => set OPENAI_API_KEY is done
mykey => set LANGCHAIN_API_KEY is done
mykey => set LANGCHAIN_HUB_API_KEY is done
mykey => set GOOGLE_API_KEY is done
mykey => set GOOGLE_CSE_ID is done
mykey => set UPSTAGE_API_KEY is done
mykey => set COHERE_API_KEY is done
mykey => set JINA_API_KEY is done
mykey => set ANTHROPIC_API_KEY is done
mykey => set DEEPL_API_KEY is done
mykey => set TAVILY_API_KEY is done
mykey => set TOGETHER_API_KEY is done


In [ ]:

# --- 1. OpenAI 클라이언트 설정 ---
try:
  client = openai.OpenAI() 
  if not client.api_key:
    raise openai.OpenAIError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
except Exception as e:
  print(f"API 키 로드 오류: {e}")
  API_ERROR = str(e)
else:
  API_ERROR = None

# --- 2. (메인) 챗봇 시스템 프롬프트 ---
SYSTEM_PROMPT = """
당신은 매우 친절하고 지능적인 한국 여행 도우미 챗봇입니다.
당신의 유일한 임무는 사용자와 자연스러운 대화를 나누며, 12가지 필수 정보를 수집하여 JSON 프로필을 완성하는 것입니다.

[수집해야 할 12개 항목 스키마]
1.  age: (예: "10대", "20대", "30대"...)
2.  gender: (예: "남", "여", "기타")
3.  nationality: (예: "미국", "일본", "중국")
4.  travel_type: (예: "가족", "혼자", "친구", "연인")
5.  party_size: (예: 1, 2, 4...)
6.  can_wait: (웨이팅 가능 여부, 예: "O", "X")
7.  budget: (예산 수준, 예: "저", "중", "고")
8.  spicy_ok: (매운 음식 가능 여부, 예: "O", "X")
9.  is_vegetarian: (채식 여부, 예: "O", "X")
10. avoid_ingredients: (절대 불가 식재료, 예: "돼지고기", "견과류", "없음")
11. like_ingredients: (좋아하는 식재료, 예: "닭고기", "해산물", "야채")
12. food_category: (선호 음식 분류, 예: "한식", "일식", "디저트", "상관없음")

[대화 규칙]
1.  대화는 당신이 먼저 시작합니다. 환영 인사와 함께 첫 질문(예: 연령대)을 하세요.
2.  항상 한 번에 하나씩만 질문하세요.
3.  사용자의 답변을 분석하여 [현재 프로필]을 업데이트합니다. (한 번에 여러 정보가 들어오면 모두 업데이트)
4.  업데이트된 프로필을 확인하고, 아직 'null'이거나 수집되지 않은 항목 중 하나를 골라 자연스럽게 다음 질문을 합니다.
5.  모든 12개 항목이 수집되면, "설문이 완료되었습니다! 감사합니다."라는 메시지를 보내고 더 이상 질문하지 마세요.
6.  매우 친절하고 공감하는 톤을 유지하세요.

[필수 출력 포맷]
당신은 *반드시* 다음 JSON 형식으로만 응답해야 합니다.
{
  "updated_profile": {
    "age": "20대", // 수집된 값 또는 이전 값
    "gender": "남",
    "nationality": null, // 아직 수집 안 됨
    // ... (12개 항목 모두 포함) ...
  },
  "bot_response": "아, 20대 남성이시군요! 반갑습니다. 혹시 국적이 어떻게 되시나요?"
}
"""

# --- 3. 12개 항목의 기본 템플릿 ---
PROFILE_TEMPLATE = {
  "age": None, "gender": None, "nationality": None, "travel_type": None,
  "party_size": None, "can_wait": None, "budget": None, "spicy_ok": None,
  "is_vegetarian": None, "avoid_ingredients": None, "like_ingredients": None,
  "food_category": None
}

# --- 4. [신규] JSON -> 구어체 변환 함수 ---
def generate_profile_summary(profile_data):
  """
  완성된 프로필(JSON)을 받아, 
  GPT-4o를 호출하여 구어체 요약문을 생성합니다.
  """
  profile_str = json.dumps(profile_data, indent=2, ensure_ascii=False)
  
  # 요약용 프롬프트 (메인 프롬프트와 역할이 다름)
  summary_system_prompt = """
  당신은 JSON 프로필 데이터를 받아서, 그 사람의 입장에서 자신을 소개하는 '구어체' 텍스트로 변환하는 글쓰기 전문가입니다.
  데이터를 기반으로 매우 자연스럽고 친근한 톤으로 "안녕하세요! 저는..." 하고 시작하는 1인칭 자기소개 글을 작성해주세요.
  
  [규칙]
  1. 1인칭 시점("저는", "제가")을 사용하세요.
  2. 딱딱한 리스트가 아닌, 하나의 연결된 문단으로 만드세요.
  3. 모든 정보를 포함하되, 자연스럽게 문장에 녹여내세요. (예: 'spicy_ok: O' -> '매운 음식도 잘 먹는 편이에요.')
  4. 'party_size'와 'travel_type'을 묶어서 (예: "이번엔 가족(친구)이랑 O명이서 왔어요")처럼 표현하세요.
  5. 'budget'은 "가성비 있는(저렴한)", "적당한", "고급스러운" 등으로 표현하세요.
  6. 'can_wait'는 "맛집이라면 줄 서는 것도 괜찮아요" 등으로 표현하세요.
  """
  
  user_prompt = f"""
  [사용자 프로필 JSON]
  {profile_str}

  위 프로필을 바탕으로 규칙에 맞게 자기소개 글을 작성해주세요.
  """
  
  try:
    response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": summary_system_prompt},
        {"role": "user", "content": user_prompt}
      ],
      temperature=0.7
    )
    summary = response.choices[0].message.content
    # 챗봇창에 예쁘게 표시될 헤더 추가
    return f"\n\n---\n\n### 🤖 AI가 파악한 Charlie님의 프로필\n\n{summary}"
  except Exception as e:
    print(f"요약 생성 오류: {e}")
    return "\n\n(프로필 요약 생성에 실패했습니다.)"
# -----------------------------------------------


def call_gpt4o(chat_messages, current_profile):
  """(메인) GPT-4o API를 호출하고 JSON 응답을 파싱하는 함수"""
  
  system_message_with_profile = f"""
  {SYSTEM_PROMPT}

  [현재까지 수집된 프로필]
  {json.dumps(current_profile, indent=2, ensure_ascii=False)}
  
  [대화 기록]
  (대화 기록은 아래 메시지 리스트에 포함되어 있습니다)
  """
  
  messages_for_api = [
    {"role": "system", "content": system_message_with_profile}
  ]
  messages_for_api.extend(chat_messages)

  try:
    response = client.chat.completions.create(
      model="gpt-4o",
      messages=messages_for_api,
      response_format={"type": "json_object"}, 
      temperature=0.7
    )
    
    response_content = response.choices[0].message.content
    response_data = json.loads(response_content)
    
    bot_message = response_data.get("bot_response", "오류가 발생했습니다.")
    updated_profile = response_data.get("updated_profile", current_profile)
    
    return bot_message, updated_profile
    
  except Exception as e:
    print(f"API 호출 또는 JSON 파싱 오류: {e}")
    error_message = f"죄송합니다. 챗봇 응답 생성 중 오류가 발생했습니다: {e}"
    return error_message, current_profile


def start_chat():
  """
  채팅방이 처음 로드될 때 실행.
  GPT-4o를 호출하여 첫 번째 질문을 받는다.
  """
  if API_ERROR:
    return [("시스템 오류", API_ERROR)], [], PROFILE_TEMPLATE.copy(), False
    
  initial_profile = PROFILE_TEMPLATE.copy()
  
  bot_message, updated_profile = call_gpt4o(
    chat_messages=[], 
    current_profile=initial_profile
  )
  
  gradio_history = [(None, bot_message)] 
  llm_history = [{"role": "assistant", "content": bot_message}]
  
  # [수정됨] 4개 값 반환 (is_completed = False)
  return gradio_history, llm_history, updated_profile, False

def chat_survey(message, gradio_history, llm_history, current_profile, is_completed):
  """
  사용자가 메시지를 입력할 때마다 실행되는 메인 함수
  """
  if API_ERROR:
    gradio_history.append((message, f"시스템 오류로 응답할 수 없습니다: {API_ERROR}"))
    return gradio_history, llm_history, current_profile, is_completed

  # 1. 사용자 메시지를 LLM 대화 기록에 추가
  llm_history.append({"role": "user", "content": message})
  
  # 2. GPT-4o API 호출 (정보 수집)
  bot_message, updated_profile = call_gpt4o(llm_history, current_profile)
  
  # 3. 봇 응답을 LLM 대화 기록에 추가
  llm_history.append({"role": "assistant", "content": bot_message})

  # --- 5. [수정됨] 완료 여부 확인 및 요약 생성 ---
  final_bot_message = bot_message
  
  # 프로필의 모든 값이 None이 아닌지 확인
  profile_is_complete = all(v is not None for v in updated_profile.values())
  
  if profile_is_complete and not is_completed:
    # ★★★ 핵심 로직 ★★★
    # 프로필이 방금 완성되었고 (True), 
    # 아직 요약본을 생성한 적이 없다면 (False)
    
    print("--- 프로필 완성! 요약 생성을 시작합니다. ---")
    
    # Gradio UI에 로딩 메시지 표시
    gr.Info("프로필이 완성되었습니다! AI가 자기소개 요약을 생성 중입니다...")
    
    # 두 번째 GPT-4o 호출 (요약 생성)
    summary_text = generate_profile_summary(updated_profile)
    
    # 봇의 최종 응답 = "설문 완료" 메시지 + "요약문"
    final_bot_message = f"{bot_message}\n{summary_text}"
    
    # 상태 변경: 이제 완료되었음 (다음부터는 요약 생성 안 함)
    is_completed = True 
    
    # (디버깅용) 최종 요약본이 포함된 프로필
    print(json.dumps(updated_profile, indent=2, ensure_ascii=False))
    print(summary_text)

  # 6. Gradio 챗봇 기록 업데이트 (final_bot_message 사용)
  gradio_history.append((message, final_bot_message))
  
  # 7. [수정됨] 4개의 상태 반환
  return gradio_history, llm_history, updated_profile, is_completed
# -----------------------------------------------

# --- 6. Gradio UI 인터페이스 빌드 ---

with gr.Blocks(theme=gr.themes.Soft()) as demo:
  gr.Markdown("# 🤖 GPT-4o 기반 자연어 서베이 챗봇 (요약 기능)")
  gr.Markdown("AI가 12가지 프로필 정보를 수집하고, 완료되면 구어체로 요약합니다.")
  
  # 1. 보이지 않는 상태(State) 변수 선언
  llm_history_state = gr.State(value=[]) 
  profile_state = gr.State(value=PROFILE_TEMPLATE.copy())
  # [신규] 설문이 완료되었는지 확인하는 상태 (True/False)
  is_completed_state = gr.State(value=False)

  # 2. 채팅창 컴포넌트
  chatbot = gr.Chatbot(label="서베이 챗봇", height=600, show_copy_button=True)
  
  # 3. 사용자 입력 텍스트박스
  msg_textbox = gr.Textbox(label="답변 입력", placeholder="여기에 답변을 입력하고 Enter를 누르세요...")

  # 4. 앱이 처음 로드될 때 실행할 함수 연결
  demo.load(
    fn=start_chat,
    inputs=None,
    # [수정됨] 4개 상태 모두 초기화
    outputs=[chatbot, llm_history_state, profile_state, is_completed_state]
  )
  
  # 5. 사용자가 텍스트박스에서 Enter(submit)를 누를 때 실행할 함수 연결
  msg_textbox.submit(
    fn=chat_survey,
    # [수정됨] 4개 상태 모두 입력
    inputs=[msg_textbox, chatbot, llm_history_state, profile_state, is_completed_state],
    # [수정됨] 4개 상태 모두 출력
    outputs=[chatbot, llm_history_state, profile_state, is_completed_state]
  )
  
  # 6. Enter 누른 후 텍스트박스 비우기
  msg_textbox.submit(lambda: "", inputs=None, outputs=msg_textbox)

# 앱 실행
if __name__ == "__main__":
  if API_ERROR:
    print(f"!!! 경고: {API_ERROR}")
    print("!!! OpenAI API 키를 확인하세요. (예: export OPENAI_API_KEY=sk-...)")
  demo.launch()

/var/folders/sj/3r49bs6178v7nfhcxh7lpzd80000gn/T/ipykernel_85157/3704666468.py:234: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="서베이 챗봇", height=600, show_copy_button=True)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


--- 프로필 완성! 요약 생성을 시작합니다. ---
{
  "age": "50대",
  "gender": "남",
  "nationality": "영국",
  "travel_type": "혼자",
  "party_size": 1,
  "can_wait": "O",
  "budget": "중",
  "spicy_ok": "X",
  "is_vegetarian": "X",
  "avoid_ingredients": "없음",
  "like_ingredients": "소고기",
  "food_category": "한식"
}


---

### 🤖 AI가 파악한 Charlie님의 프로필

안녕하세요! 저는 50대 영국 남성입니다. 이번엔 혼자 여행을 왔어요. 저는 한식을 좋아해서 한국의 다양한 음식을 경험하고 싶어요. 매운 음식은 조금 피하지만, 소고기가 들어간 요리는 정말 좋아한답니다. 비건은 아니어서 다양한 재료를 즐길 수 있어요. 여행 중에는 적당한 예산으로 맛있는 음식을 찾아다니는 걸 좋아하고, 맛집이라면 줄 서는 것도 괜찮다고 생각해요. 한국의 맛있는 음식을 찾아다니는 이 여정이 정말 기대됩니다!


In [11]:
demo.close()

Closing server running on port: 7860


In [ ]:
import openai
import json
import os
import time

# --- 1. 설정 ---
# (사용자 설정값 반영)
NUM_PROFILES = 500
SLEEP_TIMER = 0.5
OUTPUT_FILE = f"synthetic_summaries_{NUM_PROFILES}.txt"
GPT_API_NAME = "gpt-4.1"
BATCH_SIZE = 25 

# --- 2. OpenAI 클라이언트 설정 ---
try:
  client = openai.OpenAI()
  if not client.api_key:
    raise openai.OpenAIError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
except Exception as e:
  print(f"API 키 로드 오류: {e}")
  print("환경 변수(OPENAI_API_KEY)를 설정했는지 확인하세요.")
  exit()

# --- 3. 프롬프트 정의 (3단계) ---

# (★★★ 수정됨 ★★★) [1차 호출] *배치* 프로필 생성 프롬프트
# .format() 오류를 피하기 위해 { 와 } 를 {{ 와 }} 로 이스케이프
# AI가 리스트가 아닌 'profiles' 키를 포함한 JSON 객체를 반환하도록 수정
BASE_PROFILE_BATCH_PROMPT = """
당신은 현실적이고 다양한 '사용자 기본 profil'을 JSON 객체로 생성하는 AI입니다.
[논리 규칙]을 엄격하게 준수해야 합니다.

[요청]
- [6개 스키마]를 따르는 {batch_size}개의 *고유하고 중복되지 않는* 사용자 프로필을 생성해주세요.
- (필수) 응답은 오직 'profiles'라는 *단일 키*를 가진 JSON 객체여야 합니다.
- 'profiles'의 값은 생성된 {batch_size}개의 프로필 JSON 객체 *리스트*여야 합니다.

[응답 예시]
{{
  "profiles": [
    {{ "name": "John", "age": "20대", ... }},
    {{ "name": "Yuki", "age": "30대", ... }}
  ]
}}

[6개 스키마 (프로필 1개당)]
{{
  "name": "(성별과 국적에 어울리는 현실적이고 *독특한* 이름)",
  "age": "(10대, 20대, 30대, 40대, 50대, 60대+)",
  "gender": "(남, 여)",
  "nationality": "(다양한 국적. 예: 미국, 일본, 독일, 브라질 등)",
  "travel_type": "(혼자, 가족, 친구, 연인)",
  "party_size": (숫자, 1~8)
}}

[논리 규칙]
1. (필수) name은 gender와 nationality에 어울려야 하며, 매우 다양하게 생성해야 합니다.
2. (필수) travel_type이 '혼자'이면 party_size는 반드시 1이어야 합니다.
3. (필수) travel_type이 '연인'이면 party_size는 반드시 2여야 합니다.
4. (필수) travel_type이 '가족'이나 '친구'이면 party_size는 2 이상 8 이하 여야 합니다.
"""

# [2차 호출] 음식 프로필 생성 프롬프트 (food_category 포함 7개)
FOOD_PROFILE_PROMPT = """
당신은 [사용자 기본 프로필]을 바탕으로, 그 사람에게 어울리는 '음식 선호도'를 JSON 객체로 생성하는 AI입니다.
[7개 스키마]와 [논리 규칙]을 엄격하게 준수해야 합니다.
응답은 오직 7개 스키마를 포함한 JSON 객체(코드 블록 제외)여야 합니다.

[7개 스키마]
{
  "can_wait": "(O, X)",
  "budget": "(저, 중, 고)",
  "spicy_ok": "(O, X)",
  "is_vegetarian": "(O, X)",
  "avoid_ingredients": "(문자열, 예: '견과류', '없음', '돼지고기')",
  "like_ingredients": "(문자열, 예: '소고기', '해산물', '야채')",
  "food_category": "(일식, 중식, 한식, 양식, 동남아, 디저트류, 상관없음)"
}

[논리 규칙]
1. (필수) [기본 프로필]의 'age'가 '60대+'이거나 'budget'이 '고'이면 'can_wait'는 'X'일 확률이 높습니다.
2. (필수) [기본 프로필]의 'nationality'가 멕시코, 인도, 베트남 등이면 'spicy_ok'는 'O'일 확률이 높습니다.
3. (필수) is_vegetarian이 'O'이면, like_ingredients는 절대 "소고기", "돼지고기" 등이 될 수 없습니다. (예: "두부", "야채")
4. (필수) food_category는 [기본 프로필]의 'nationality'와 'like_ingredients'를 고려하여 논리적으로 설정합니다.
"""

# [3차 호출] 요약문 생성 프롬프트 (13개 항목 기준)
SUMMARY_GENERATION_PROMPT = """
당신은 *완성된* 13개 항목(기본 6 + 음식 7)의 JSON 프로필 데이터를 받아서, 그 사람의 입장에서 자신을 소개하는 '구어체' 텍스트로 변환하는 글쓰기 전문가입니다.
데이터를 기반으로 매우 자연스럽고 친근한 톤으로 1인칭 자기소개 글을 작성해주세요.

[규칙]
1. (필수) JSON의 'name' 필드를 사용하여 "안녕하세요! 저는 [name]입니다."로 문장을 시작하세요.
2. 1인칭 시점("저는", "제가")을 사용하세요.
3. (필수) 13개 항목의 모든 정보를 포함하되, 자연스럽게 문장에 녹여내세요.
4. 'party_size'와 'travel_type'을 묶어서 (예: "이번엔 가족이랑 4명이서 왔어요")처럼 표현하세요.
"""

# --- 4. 헬퍼 함수 정의 (3단계) ---

# (★★★ 수정됨 ★★★) 1차 호출: 배치 생성 함수
def generate_base_profile_batch(batch_size, start_id_num):
  """ [1차 호출] 기본 프로필을 'batch_size'만큼 생성 """
  print(f"  > [1/3] ID {start_id_num}~{start_id_num + batch_size - 1}: {batch_size}개 기본 프로필 *배치* 생성 중...")
  
  # 프롬프트에 'batch_size' 변수 삽입
  prompt_content = BASE_PROFILE_BATCH_PROMPT.format(batch_size=batch_size)
  
  try:
    response = client.chat.completions.create(
      model=GPT_API_NAME, 
      messages=[
        {"role": "system", "content": prompt_content},
        {"role": "user", "content": f"규칙에 맞는 {batch_size}개의 고유한 사용자 기본 프로필 JSON *객체*를 [응답 예시] 형식에 맞게 생성해주세요."}
      ],
      response_format={"type": "json_object"},
      temperature=1.0 
    )
    # (★★★ 수정됨 ★★★) AI의 응답은 이제 'profiles' 키를 가진 JSON 객체임
    response_json = json.loads(response.choices[0].message.content)
    
    # 'profiles' 키에서 리스트를 추출
    batch_profiles_list = response_json.get("profiles")

    # (★★★ 수정됨 ★★★) 검증 로직
    if not isinstance(batch_profiles_list, list):
      print(f"  > [오류] 1차(배치) 생성 실패: AI가 'profiles' 키에 리스트(list)를 반환하지 않았습니다.")
      return None
      
    if len(batch_profiles_list) != batch_size:
      print(f"  > [오류] 1차(배치) 생성 실패: AI가 {batch_size}개가 아닌 {len(batch_profiles_list)}개를 반환했습니다.")
      return None
      
    # (★★★ 수정됨 ★★★) 리스트 자체를 반환
    return batch_profiles_list
  
  except Exception as e:
    print(f"  > [오류] 1차(배치) 생성 실패: {e}")
    return None

# (이하 함수는 1:1 처리로 동일)
def generate_food_profile(base_profile_data):
  """ [2차 호출] 1차 프로필에 맞는 음식 정보 생성 """
  user_name = base_profile_data.get('name', 'N/A')
  print(f"  > [2/3] {user_name}: 음식 프로필 생성 중...")
  base_profile_str = json.dumps(base_profile_data, indent=2, ensure_ascii=False)
  try:
    response = client.chat.completions.create(
      model=GPT_API_NAME,
      messages=[
        {"role": "system", "content": FOOD_PROFILE_PROMPT},
        {"role": "user", "content": f"[사용자 기본 프로필]\n{base_profile_str}\n\n위 사용자의 음식 선호도를 [논리 규칙]에 맞게 JSON으로 생성해주세요."}
      ],
      response_format={"type": "json_object"},
      temperature=0.8
    )
    food_profile_json = json.loads(response.choices[0].message.content)
    return food_profile_json
  except Exception as e:
    print(f"  > [오류] 2차(음식) 생성 실패: {e}")
    return None

def generate_profile_summary(full_profile_data):
  """ [3차 호출] 완성된 JSON으로 구어체 요약문 생성 """
  user_name = full_profile_data.get('name', 'N/A')
  print(f"  > [3/3] {user_name}: 최종 요약문 생성 중...")
  full_profile_str = json.dumps(full_profile_data, indent=2, ensure_ascii=False)
  try:
    response = client.chat.completions.create(
      model=GPT_API_NAME,
      messages=[
        {"role": "system", "content": SUMMARY_GENERATION_PROMPT},
        {"role": "user", "content": f"[완성된 사용자 프로필 JSON]\n{full_profile_str}\n\n위 프로필을 바탕으로 규칙에 맞게 자기소개 글을 작성해주세요."}
      ],
      temperature=0.7
    )
    summary = response.choices[0].message.content
    return summary
  except Exception as e:
    print(f"  > [오류] 3차(요약) 생성 실패: {e}")
    return None

# --- 5. 메인 스크립트 실행 ---
def main():
  print(f"총 {NUM_PROFILES}개의 합성 데이터 생성을 시작합니다...")
  print(f"모델: {GPT_API_NAME}, 배치 크기: {BATCH_SIZE}")
  
  # --- 1단계: 기본 프로필 *배치* 생성 ---
  print(f"\n--- 1단계: {NUM_PROFILES}개의 기본 프로필 (배치 크기 {BATCH_SIZE}) 생성 시작 ---")
  all_base_profiles = []
  
  # BATCH_SIZE 단위로 루프 실행
  for i in range(0, NUM_PROFILES, BATCH_SIZE):
    # 현재 배치의 실제 크기 계산
    current_batch_size = min(BATCH_SIZE, NUM_PROFILES - i)
    current_start_id = i + 1
    
    print(f"--- 1단계 배치 {i//BATCH_SIZE + 1}/{ -(-NUM_PROFILES // BATCH_SIZE)} (ID {current_start_id}부터 {current_start_id + current_batch_size - 1}까지) ---")
    
    # 배치 생성 함수 호출
    batch_result_list = generate_base_profile_batch(current_batch_size, current_start_id)
    
    if batch_result_list:
      # 배치 결과(리스트)에 ID를 추가하며 all_base_profiles에 저장
      for j, profile in enumerate(batch_result_list):
        profile["id"] = f"user_{current_start_id + j:04d}"
        all_base_profiles.append(profile)
    else:
      print(f"  > [경고] {current_start_id}번부터의 배치 생성을 건너뜁니다.")
    
    # 배치 호출 사이에만 대기
    time.sleep(SLEEP_TIMER)

  print(f"\n--- 1단계 완료: 총 {len(all_base_profiles)}개의 기본 프로필 생성 ---")
  
  
  # --- 2단계: 음식 정보 및 요약문 생성 (개별 순회) ---
  print(f"\n--- 2단계: 음식 정보/요약문 개별 생성 시작 ---")
  all_summaries_list = []
  num_generated = len(all_base_profiles)

  for i, base_profile in enumerate(all_base_profiles):
    user_id = base_profile.get("id")
    user_name = base_profile.get("name", "N/A")
    print(f"\n--- 2단계 작업 {i+1}/{num_generated} (ID: {user_id}, 이름: {user_name}) ---")
    
    # 2. [2차 호출] 음식 프로필 생성
    food_profile = generate_food_profile(base_profile)
    if food_profile is None:
      print(f"  > [경고] {user_id}의 2차 생성을 건너뜁니다.")
      continue
    # 두 번째 API 호출 후 대기
    time.sleep(SLEEP_TIMER)

    # 3. 데이터 합치기
    full_profile = {**base_profile, **food_profile}
    
    # 4. [3차 호출] 최종 요약문 생성
    summary_text = generate_profile_summary(full_profile)
    if summary_text is None:
      print(f"  > [경고] {user_id}의 3차 생성을 건너뜁니다.")
      continue
      
    # 5. 결과 리스트에 요약문만 추가
    formatted_summary = f"## {i+1}. {user_name} ({user_id})\n\n{summary_text}\n\n---\n"
    all_summaries_list.append(formatted_summary)
    
    # 세 번째 API 호출 후 대기
    time.sleep(SLEEP_TIMER)

  # 6. 텍스트 파일로 저장
  try:
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
      f.write("\n".join(all_summaries_list))
      
    print(f"\n--- 작업 완료! ---")
    print(f"총 {len(all_summaries_list)}개의 '요약문'이 '{OUTPUT_FILE}' 파일에 저장되었습니다.")
  except Exception as e:
    print(f"\n[치명적 오류] 파일 저장 실패: {e}")

if __name__ == "__main__":
  main()

총 500개의 합성 데이터 생성을 시작합니다...
모델: gpt-4.1, 배치 크기: 25

--- 1단계: 500개의 기본 프로필 (배치 크기 25) 생성 시작 ---
--- 1단계 배치 1/20 (ID 1부터 25까지) ---
  > [1/3] ID 1~25: 25개 기본 프로필 *배치* 생성 중...
--- 1단계 배치 2/20 (ID 26부터 50까지) ---
  > [1/3] ID 26~50: 25개 기본 프로필 *배치* 생성 중...
--- 1단계 배치 3/20 (ID 51부터 75까지) ---
  > [1/3] ID 51~75: 25개 기본 프로필 *배치* 생성 중...
--- 1단계 배치 4/20 (ID 76부터 100까지) ---
  > [1/3] ID 76~100: 25개 기본 프로필 *배치* 생성 중...
--- 1단계 배치 5/20 (ID 101부터 125까지) ---
  > [1/3] ID 101~125: 25개 기본 프로필 *배치* 생성 중...
--- 1단계 배치 6/20 (ID 126부터 150까지) ---
  > [1/3] ID 126~150: 25개 기본 프로필 *배치* 생성 중...
--- 1단계 배치 7/20 (ID 151부터 175까지) ---
  > [1/3] ID 151~175: 25개 기본 프로필 *배치* 생성 중...
--- 1단계 배치 8/20 (ID 176부터 200까지) ---
  > [1/3] ID 176~200: 25개 기본 프로필 *배치* 생성 중...
--- 1단계 배치 9/20 (ID 201부터 225까지) ---
  > [1/3] ID 201~225: 25개 기본 프로필 *배치* 생성 중...
--- 1단계 배치 10/20 (ID 226부터 250까지) ---
  > [1/3] ID 226~250: 25개 기본 프로필 *배치* 생성 중...
--- 1단계 배치 11/20 (ID 251부터 275까지) ---
  > [1/3] ID 251~275: 25개 기본 프로필 *배치* 생성 중...
--- 1단